### <center>Machine Learning HW1
**<center> Name: Tongxuan Tian, Computing ID: nua3jz, Date: 09/21/2023**

#### Problem 1 Solution
To prove that Bayes predictor is the optimal predictor, we need to prove that for any predictor $h$, we have:
$$L_D(f_D) \leq L_D(f_h)$$
Since $L_D(f_D) = P_{x, y \in D}(h(x) \neq y)$, we can write is as:
$$ L_D(f_D)  =\begin{cases}
 \text{$\mathbb{P}[y=+1|x]$ if $h(x)=+1$} \;\;\\
 \text{$\mathbb{P}[y=-1|x]$ if $h(x)=-1$} \;\;
\end{cases}$$
Since it is a binary classification problem, we can know that $ \mathbb{P}[y=+1|x]+\mathbb{P}[y=-1|x] = 1 $, thus we have:
$$ L_D(f_D)  =\begin{cases}
 \text{$\mathbb{P}[y=+1|x]$     if $h(x)=+1$} \;\;\\
 \text{$1 - \mathbb{P}[y=+1|x]$ if $h(x)=-1$} \;\;
\end{cases}$$  
Then we should discuss on a case-by-case basis.
1. When $ \mathbb{P}[y=+1|x] < 1 - \mathbb{P}[y=-1|x] $, we have $\mathbb{P}[y=+1|x] < \frac{1}{2}$ and $\mathbb{P}[y=-1|x] > \frac{1}{2}$, which means $f_D(x) = -1$.  
Then we have $L_D(f_D) = \mathbb{P}[y=+1|x] < \frac{1}{2}$. Thus, we can get that $L_D(f_D) \leq L_D(f_h)$.
2. Similarly, when $ \mathbb{P}[y=+1|x] > 1 - \mathbb{P}[y=-1|x] $, we have $\mathbb{P}[y=+1|x] > \frac{1}{2}$ and $\mathbb{P}[y=-1|x] < \frac{1}{2}$, which means $f_D(x) = +1$.  
Then we have $L_D(f_D) = \mathbb{P}[y=-1|x] < \frac{1}{2}$. Thus, we can get that $L_D(f_D) \leq L_D(f_h)$.  

Based on above, we can know that for any $h$, we always have $L_D(f_D) \leq L_D(f_h)$.


#### Problem 2 Solution
**Question (a)**  
It is obvious that the decision boundary of this Bayes predictor is the intersection point of 2 normal distribution. Thus, we can get it through the equation below:
$$ \frac{1}{2}N(x;0,1) = \frac{1}{2}N(x;\frac{2}{3}\pi, 0.5) $$  
We can extend it as:  
$$ \frac{1}{\sqrt{2\pi}}e^{-\frac{x^2}{2}} = \frac{1}{0.5\sqrt{2\pi}}e^{-\frac{(x-\frac{2}{3}\pi)^2}{2(0.5)^2}} $$
And we can get the solution to this equality is $ x \approx 1.2396 $, which is the decision boundary $b_{Bayes}$ we want for this Bayes predictor.  

**Question (b)**  
The true error of this Bayes predictor is actually the intersection area of two Gaussian distributions. Thus, we can written the true error as:  
$$ L_{D}(f_D) = \int_{-\infty}^{b_{Bayes}} \frac{1}{2}N(x;0,1) dx + \int_{b_{Bayes}}^{+\infty} \frac{1}{2}N(x;\frac{2}{3}\pi, 0.5) dx$$  
We can easily get that $$ L_{D}(f_D) \approx 0.0756 $$  
Thus, the true error of this Bayes predictor $ L_{D}(f_D) $ is 0.0756  

**Question (c)**  
To find the best hypothesis $ h^* \in H $, we need to find the minimal true error. Run code block below to get the best hypothesis and corresponding decision boundary (May take about 30s~).

In [1]:
import numpy as np
from scipy.stats import norm
from scipy.integrate import quad
from scipy.optimize import fsolve

h_num = 1200

def find_intsec(mu1, sigma1, mu2, sigma2):
    pdf1 = lambda x: norm.pdf(x, loc=mu1, scale=sigma1)
    pdf2 = lambda x: norm.pdf(x, loc=mu2, scale=sigma2)
    
    intersection_func = lambda x: pdf1(x) - pdf2(x)
    intersection_x = fsolve(intersection_func, (mu1 + mu2) / 2)
    
    return intersection_x


def integrate_normal(mu, sigma, lower_limit, upper_limit):
    pdf = lambda x: norm.pdf(x, loc=mu, scale=sigma)
    result, _ = quad(pdf, lower_limit, upper_limit)
    
    return result


def find_best_hypothsis():
    min_error = np.inf
    best_h = 1
    for i in range(1, h_num + 1):
        h = i / 400
        
        true_error = 0.5 * integrate_normal((2/3)*np.pi, 0.5, -np.inf, h) \
            + 0.5 * integrate_normal(0, 1, h, np.inf)

        if true_error < min_error:
            min_error = true_error
            best_h = i

    return best_h, min_error

if __name__ == "__main__":
    best_h, min_error = find_best_hypothsis()

    print(f"Best hypothesis i = {best_h}")
    print(f"Decision boundary b = {best_h / 400}")
    print(f"True error L = {min_error}")

Best hypothesis i = 496
Decision boundary b = 1.24
True error L = 0.07561630298491684


**Question (d)**  
Based on the result of the above code block, we can know that true error $L_D(h^*) \approx 0.0756$  

**Question (e)**  
Similarly, we need to minimize the empirical error on data we generated to find the best hypothesis $h_S$.  Run code block below to get the best hypothesis and corresponding decision boundary.

In [6]:
from numpy.random import normal

N = 100                 # sample size
comp_idx = [1,-1]       # lable
x_pos, x_neg = [], []   # observations

for i in range(N):
    x_neg.append(normal(0, 1))
    x_pos.append(normal((2 / 3) * np.pi, 0.5))

min_error = 1
best_h = 1
for i in range(1, h_num + 1):
        
    h = i / 400
    em_error = 0

    for x_p in x_pos:
        if x_p <= h:
            em_error += 1
    
    for x_n in x_neg:
        if x_n >= h:
            em_error += 1

    em_risk  = em_error / (2 * N)

    if em_risk < min_error:
        min_error = em_risk
        best_h = i

print(f"Best hypothesis: i = {best_h}")
print(f"Decision boundary: bs = {best_h / 400}")

Best hypothesis: i = 500
Decision boundary: bs = 1.25


**Question (f)**  
Similar to **question (b)**, since we know the decision boundary $b_S$ of this hypothesis, we written its corresponding true error as:  
$$ L_D(h_S)= \int_{-\infty}^{b_S} \frac{1}{2}N(x;0,1) dx + \int_{b_S}^{+\infty} \frac{1}{2}N(x;\frac{2}{3}\pi, 0.5) dx$$  
Run code block below to get the true error.

In [7]:
true_error = 0.5 * integrate_normal((2 / 3)*np.pi, 0.5, -np.inf, best_h / 400) \
    + 0.5 * integrate_normal(0, 1, best_h / 400, np.inf)
print(f"True error: L = {true_error}")

True error: L = 0.07563979723904254


#### Problem 3 Solution
Code block below is the python implementation of Perceptron Algorithm. Run it to get the final $w^{(t)}$. Make sure **data.txt** is in the same folder of this jupyter notebook file.

In [12]:
import numpy as np

def read_txt(file_path):
    
    file = open(file_path,'r')
    file_data = file.readlines()
    x1, x2, y = [], [], []

    for row in file_data:
        tmp_list = row.replace('\n', '').split('\t')
        x1.append(float(tmp_list[0]))
        x2.append(float(tmp_list[1]))
        y.append(float(tmp_list[-1]))

    return {'x1': x1, 'x2': x2,}, {'y': y}

def perception(input, lable):
    assert len(input['x1']) == len(lable['y'])
    x1 = np.array(input['x1'])
    x2 = np.array(input['x2'])

    x = np.column_stack((np.transpose(x1), np.transpose(x2),
                          np.transpose(np.ones_like(x2))))

    y = np.array(lable['y'])
    w = np.zeros_like(x[0]) 

    while True:
        flag = 0
        for t in range(len(x)):
            i = t % len(x)
            
            if (y[i] * np.sum(w * x[i])) <= 0:
                w = w + y[i] * x[i]
            
            if (y * np.sum(w * x, axis=1) < 0).sum() <= 0:
                flag = 1
                break
        if flag:
            break

    return w


if __name__ == '__main__':
    data_path = './data.txt'
    input, lable = read_txt(data_path)
    w = perception(input=input, lable=lable)
    print(f"w1 = {w[0]}, w2 = {w[1]}, b = {w[2]}")

w1 = -3.5199999999999987, w2 = -1.2400000000000002, b = 2.0


Thus, the final $w^{(t)}$ is $[-3.52, -1.24]$ and bias is $2$

#### Problem 4 Solution
The gradient of $L(h_w, S)$ with respect to $w$ is
$$ \frac{dL(h_w, S)}{dw} = \frac{1}{m} \sum_{i=1}^{n} \frac{  \frac{d(1+exp(-y_i\langle{w}, x_i\rangle))}{dw}}{(1+exp(-y_i\langle{w}, x_i\rangle)) \ln {e}}
$$  
$$=\frac{1}{m} \sum_{i=1}^{n} \frac{exp(-y_i\langle{w}, x_i\rangle)}{(1+exp(-y_i\langle{w}, x_i\rangle)) \ln {e}} \frac{d(-y_i\langle{w}, x_i\rangle)}{dw}
$$  
$$=\frac{1}{m} \sum_{i=1}^{n} \frac{exp(-y_i\langle{w}, x_i\rangle)}{(1+exp(-y_i\langle{w}, x_i\rangle)) \ln {e}} (-y_i x_i)$$

#### Problem 5 Solution
Since $L_{l2}(h_w, S)$ is a convex function, the optimal solution can be given by the following equation:  
$$ \frac{dL_{l2}(h_w, S)}{dw}=0 $$ 
$$2\sum_{i=1}^{n}(\langle{w_i}, x_i\rangle-y_i)x_i+2 \lambda w=0$$
$$w(\sum_{i=1}^{n}(x_i x_i^T)+ \lambda)=\sum_{i=1}^{n}{y_i x_i}$$  
$$w=(\sum_{i=1}^{n}(x_i x_i^T)+ \lambda)^{-1}\sum_{i=1}^{n}{y_i x_i}$$  
By denoting $(\sum_{i=1}^{n}(x_i x_i^T)+ \lambda)^{-1} = A$ and $b = \sum_{i=1}^{n}{y_i x_i}$, we can rewrite the equality above as below in short:  
$$w=(A+ \lambda I)^{-1}b$$